In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

from src.const import SEED
from src.evaluation import eval_acc_in_binary_task
from src.preprocess import transform_to_data_loader
from src.preprocess_utils import create_binary_labels
from src.train import train_model
from src.visualization import plot_conf_matrix, plot_losses

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
class Unknown_BiLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers=4, units=512, dropout_rate=0.3):
        torch.manual_seed(SEED)
        random.seed(SEED)
        np.random.seed(SEED)

        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            batch_first=True, 
            bidirectional=True, 
            num_layers=num_layers,
            dropout=dropout_rate,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(2*hidden_size, units)
        self.fc2 = nn.Linear(units, 1)
        self.bc1 = nn.BatchNorm1d(2*hidden_size)
        self.bc2 = nn.BatchNorm1d(units)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.lstm(x)[0][:, -1, :]
        out = self.dropout(out)
        out = self.bc1(out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.bc2(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [7]:
class Silence_GRU(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers=4, units=64, dropout_rate=0.3):
        torch.manual_seed(SEED)
        random.seed(SEED)
        np.random.seed(SEED)

        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size, 
            hidden_size=hidden_size, 
            batch_first=True,
            num_layers=num_layers,
            dropout=dropout_rate,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size, units)
        self.fc2 = nn.Linear(units, 1)
        self.bc1 = nn.BatchNorm1d(hidden_size)
        self.bc2 = nn.BatchNorm1d(units)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.gru(x)[0][:, -1, :]
        out = self.dropout(out)
        out = self.bc1(out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.bc2(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [9]:
class Main_GRU(nn.Module):

    def __init__(self, input_size, hidden_size, num_class, dropout, num_layers):

        GRU_SEED=420
        torch.manual_seed(GRU_SEED)
        random.seed(GRU_SEED)
        np.random.seed(GRU_SEED)

        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size, 
            hidden_size=hidden_size, 
            batch_first=True,
            num_layers=num_layers,
            dropout=dropout,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, x):
        out = self.gru(x)[0][:, -1, :]
        out = self.fc(out)
        return out

In [6]:
input_size = 20
hidden_size = 32
num_layers = 4

unknown_bilstm_model = Unknown_BiLSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
unknown_bilstm_model.load_state_dict(torch.load("unknown_best_model"))

<All keys matched successfully>

In [8]:
input_size = 20
hidden_size = 16
num_layers = 2

silence_gru_model = Silence_GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
silence_gru_model.load_state_dict(torch.load("silence_best_model"))

<All keys matched successfully>

In [11]:
input_size = 20
hidden_size = 64
dropout = 0.5
num_layers = 4
num_class = 10

main_gru_model = Main_GRU(input_size, hidden_size, num_class, dropout, num_layers).to(device)
main_gru_model.load_state_dict(torch.load("main_best_model"))

<All keys matched successfully>